In [30]:
import requests
import time
import pandas as pd 
import numpy as np 

# textrank
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

from selenium import webdriver 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re

from pymongo import MongoClient 
import os

def conn_db():
    user = os.getenv("DBUSER", 'signature')
    pwd = os.getenv("DBPWD", 'shanekang')
    ip_addr = os.getenv("DBADDR", '54.180.213.105')
    conn = MongoClient(f'mongodb://{user}:{pwd}@{ip_addr}:27017') 
    return conn

def insert_brefing_data(doc, collection, reset=True): 
    
    if reset: collection.drop() 
    collection.insert(doc) 
    return True

def briefing_craw():

    url = 'https://www.korea.kr/special/policyFocusList.do'

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")

    browser = webdriver.Chrome('chromedriver', chrome_options=options) # 같은 디렉토리 내에 chtromdriver.exe
    browser.implicitly_wait(1)

    browser.get(f'{url}')
    browser.implicitly_wait(1)
    
    browser.find_element_by_css_selector("""#container > div > article > div.article-content > div.list-type > ul > li:nth-child(2) > a > dl > dt""").click()
    
    title = browser.find_element_by_css_selector("""#container > div > article > div.article-head > h1""").text.strip() 
    date = browser.find_element_by_css_selector("""#container > div > article > div.article-head > div.info > span""").text.strip()
    
    texts = browser.find_element_by_css_selector("""#container > div > article > div.article-content""").text
    browser.close()
    
    texts = re.sub('[□○※【】]', '', texts)
    texts = texts.replace('▲', '')
    texts = texts.replace('▸', ', ')
    
    # 2. textrank
    class SentenceTokenizer(object):
    
        def __init__(self):
            self.kkma = Kkma()
            self.twitter = Twitter()
            self.stopwords = ['머니S', '머니투데이', '구독', '파이낸셜뉴스', '헬스조선', '중앙일보', '프레시안', '조선일보', '.com',
                 '매일경제', '한국', '엠빅', '뉴시스', '무단전재재', '오마이뉴스', '연합뉴스', '연합뉴스TV', 'JTBC','SBS',
                  'MBC','KBS','한겨레','YTN','MBN','TV조선','채널A','동아일보','뉴스1', '강원일보','경향신문','세계일보',
                  '한국일보','서울신문','노컷뉴스','이데일리', 'CBS', 'co.kr', '자료사진', '기자', '언론사', '섹션', '무단',
                 '무단전재재', '재배포', '재배포금지','재판매 및 DB금지', '서울경제', '재배포 금지', '운영']

            
        # Kkma를 이용하여 문장단위로 나눈 후 setences를 return
        def url2sentences(self, url):
            article = Article(url, language='ko')
            article.download()
            article.parse()
            sentences = self.kkma.sentences(article.text)

            for idx in range(0, len(sentences)):
                if len(sentences[idx]) <= 10:
                    sentences[idx-1] += (' ' + sentences[idx])
                    sentences[idx] = ''

            return sentences

        def text2sentences(self, text):
            sentences = self.kkma.sentences(text)
            for idx in range(0, len(sentences)):
                if len(sentences[idx]) <= 10:
                    sentences[idx-1] += (' ' + sentences[idx])
                    sentences[idx] = ''

            return sentences

        # sentences를 받아 twitter.nouns()를 이용하여 명사를 추출하고 nouns를 return
        def get_nouns(self, sentences):
            nouns = []
            for sentence in sentences:
                if sentence is not '':
                    nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                            if noun not in self.stopwords and len(noun) > 1]))

            return nouns

        
    # TF-IDF 모델, 그래프 생성과정을 GraphMatrix 클래스로 구현
    class GraphMatrix(object):
        def __init__(self):
            self.tfidf = TfidfVectorizer()
            self.cnt_vec = CountVectorizer()
            self.graph_sentence = []

        def build_sent_graph(self, sentence):
            tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
            self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
            return  self.graph_sentence

        def build_words_graph(self, sentence):
            cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
            vocab = self.cnt_vec.vocabulary_
            return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
        
        
    # textrank -> 가중치그래프를 이용하여 적용 -> 높은 순으로 정렬 -> 요약할 단어 갯수만큼 출력

    class Rank(object):
        def get_ranks(self, graph, d=0.85): # d = damping factor
            A = graph
            matrix_size = A.shape[0]
            for id in range(matrix_size):
                A[id, id] = 0 # diagonal 부분을 0으로
                link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
                if link_sum != 0:
                    A[:, id] /= link_sum
                A[:, id] *= -d
                A[id, id] = 1

            B = (1-d) * np.ones((matrix_size, 1))
            ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
            return {idx: r[0] for idx, r in enumerate(ranks)}
        
        
    class TextRank(object):
        def __init__(self, text):
            self.sent_tokenize = SentenceTokenizer()

            if text[:5] in ('http:', 'https'):
                self.sentences = self.sent_tokenize.url2sentences(text)
            else:
                self.sentences = self.sent_tokenize.text2sentences(text)

            self.nouns = self.sent_tokenize.get_nouns(self.sentences)

            self.graph_matrix = GraphMatrix()
            self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
            self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

            self.rank = Rank()
            self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
            self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

            self.word_rank_idx = self.rank.get_ranks(self.words_graph)
            self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

        # 요약문장 3문장
        def summarize(self, sent_num=3):
            summary = []
            index=[]
            for idx in self.sorted_sent_rank_idx[:sent_num]:
                index.append(idx)
            index.sort()
            for idx in index:
                summary.append(self.sentences[idx])

            return summary

    url = texts
    textrank = TextRank(url)
    results =  textrank.summarize(3)

    doc = dict( 
        contents=results, 
        title=title, 
        date=date 
    )

    return doc

briefing_craw()

c:\users\playdata\.conda\envs\env-01\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: use options instead of chrome_options
c:\users\playdata\.conda\envs\env-01\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


{'contents': ['강도 태 중앙 재난안전대책본부 1 총괄 조정관( 보건복지 부 2차 관) 은 1일 “ 코로나 19가 턱밑까지 와 있는 긴급한 상황” 이라고 우려했다.',
  '강 1 총괄 조정관은 이날 코로나 19 중앙 재난안전대책본부 회의를 주재한 자리에서 “ 코로나 19 일일 확 진자 수가 6 일째 400~500 명대로 좀처럼 진정되지 않고 있다 ”며 이같이 밝혔다.',
  '강도 태 1 총괄 조정관( 보건복지 부 2차 관) 이 중앙 재난안전대책본부 회의를 주재하며 발언하고 있다.'],
 'title': '정부 “코로나19 우리 턱밑까지 와 있는 긴급한 상황”',
 'date': '2020.12.01'}

In [32]:
conn = conn_db() 
brefing_info_collection = conn.NewsData.brefing_info 
doc = briefing_craw()
print("briefing_info data: ", insert_brefing_data(doc, brefing_info_collection, reset=True))

c:\users\playdata\.conda\envs\env-01\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: use options instead of chrome_options
c:\users\playdata\.conda\envs\env-01\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


briefing_info data:  True


c:\users\playdata\.conda\envs\env-01\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [ ]:
# 2. brefing info Part
brefing_info_collection = conn.NewsData.brefing_info 
doc = briefing_crawing() 
print("Insert brefing_info data: ", insert_brefing_data(doc, brefing_info_collection, reset=True))

In [34]:
for elem in brefing_info_collection.find({}): print(elem)

{'_id': ObjectId('5fc605a002cdb9b906397fc0'), 'contents': ['강도 태 중앙 재난안전대책본부 1 총괄 조정관( 보건복지 부 2차 관) 은 1일 “ 코로나 19가 턱밑까지 와 있는 긴급한 상황” 이라고 우려했다.', '강 1 총괄 조정관은 이날 코로나 19 중앙 재난안전대책본부 회의를 주재한 자리에서 “ 코로나 19 일일 확 진자 수가 6 일째 400~500 명대로 좀처럼 진정되지 않고 있다 ”며 이같이 밝혔다.', '강도 태 1 총괄 조정관( 보건복지 부 2차 관) 이 중앙 재난안전대책본부 회의를 주재하며 발언하고 있다.'], 'title': '정부 “코로나19 우리 턱밑까지 와 있는 긴급한 상황”', 'date': '2020.12.01'}
